In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# https://www.kaggle.com/competitions/diabetes-progression-b1

In [ ]:
# submission 1 is done without feature selection (ridge)
# submission 2 is done with feature selection (ridge)

# submission 5 is done without feature selection (lasso)
# submission 6 is done with feature selection (lasso)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [ ]:
data = pd.read_csv("/kaggle/input/diabetes-progression-b1/train.csv")
test = pd.read_csv("/kaggle/input/diabetes-progression-b1/test.csv")
ss = pd.read_csv("/kaggle/input/diabetes-progression-b1/sample_submission.csv")

In [ ]:
data.head()
# print(len(data))

In [ ]:
X = data.drop(["y"], axis=1)
y = data.loc[:, 'y']
testdata = test.copy()
# X.head()
y.head()

In [ ]:
# chi2 does NOT work for negative values
# bestfeatures = SelectKBest(score_func=chi2, k=10)

bestfeatures = SelectKBest(score_func=mutual_info_classif, k=10)
fit = bestfeatures.fit(X, y)

In [ ]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

In [ ]:
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns=['l1', 'l2']

In [ ]:
featureScores

In [ ]:
X = data.drop(["s2", "s6", "y"], axis=1)
testdata = test.drop(["s2", "s6"], axis=1)
X.head()

In [ ]:
X.isna().sum()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# used to select optimal technique

from sklearn.model_selection import cross_val_score

cross_val_score(LinearRegression(), X, y)
# cross_val_score(LogisticRegression(), X, y)
# cross_val_score(RandomForestClassifier(), X, y)

In [ ]:
from sklearn.linear_model import Lasso

lasso_lr = Lasso(alpha=1, max_iter=100, tol=0.1)
lasso_lr.fit(X_train, y_train)
lasso_lr.score(X_test, y_test)


In [ ]:
from sklearn.linear_model import Ridge

ridge_lr = Ridge(alpha=5, max_iter=100, tol=0.1)
ridge_lr.fit(X_train, y_train)
ridge_lr.score(X_test, y_test)

In [ ]:
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error

# X = X.values
# y = y.values

# ridge = Ridge(alpha=1.0)  

# k = 5

# kf = KFold(n_splits=k, shuffle=True, random_state=42)

# mse_scores = []
# final_pred = []

# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    
#     ridge.fit(X_train, y_train)
#     y_pred = ridge.predict(X_test)
    
#     mse = mean_squared_error(y_test, y_pred)
#     mse_scores.append(mse)

    
# final_pred = ridge.predict(testdata)
# average_mse = np.mean(mse_scores)
# print("Average MSE:", average_mse)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

X = X.values
y = y.values

lasso = Lasso(alpha=1.0)  

k = 5

kf = KFold(n_splits=k, shuffle=True, random_state=42)

mse_scores = []
final_pred = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

    
final_pred = lasso.predict(testdata)
average_mse = np.mean(mse_scores)
print("Average MSE:", average_mse)

In [ ]:
ids = ss.iloc[:, 0]
len(final_pred)

In [ ]:
df = pd.DataFrame({"ID" : ids,
                 "y" : final_pred})

In [ ]:
df.to_csv("day1_sub6.csv", index=False)